In [1]:
import cv2
import numpy as np
import os

In [2]:
import glob 
infected = glob.glob('cell_images/Parasitized/*.png')
uninfected = glob.glob('cell_images/Uninfected/*.png')

In [3]:
x = []
y = []
for i in infected:
    if i.endswith('png'):
        img = cv2.resize(cv2.imread(i), (224,224))
        x.append(img)
        y.append(1)
for j in uninfected:
    if j.endswith('png'):
        img = cv2.resize(cv2.imread(j), (224,224))
        x.append(img)
        y.append(0)

x = np.array(x)
y = np.array(y)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
print (x.shape)
print (y.shape)

(27558, 224, 224, 3)
(27558,)


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)

# (2) Get Data
#import tflearn.datasets.oxflower17 as oxflower17
#x, y = oxflower17.load_data(one_hot=True)

# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.2))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.2))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.2))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

# (4) Compile
model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam',metrics=['accuracy'])


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
__________

In [7]:
X_train_f, X_test, y_train_f, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_f, y_train_f, test_size=0.2, random_state=40)

In [12]:

# (5) Train
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1,validation_data=(X_val,y_val), shuffle=True)

Train on 17636 samples, validate on 4410 samples
Epoch 1/10
17636/17636 [==============================] - 21s 1ms/step - loss: 0.1115 - acc: 0.9590 - val_loss: 0.2486 - val_acc: 0.9138
Epoch 2/10
17636/17636 [==============================] - 22s 1ms/step - loss: 0.1073 - acc: 0.9622 - val_loss: 0.5376 - val_acc: 0.7501
Epoch 3/10
17636/17636 [==============================] - 22s 1ms/step - loss: 0.1025 - acc: 0.9650 - val_loss: 0.7741 - val_acc: 0.6973
Epoch 4/10
17636/17636 [==============================] - 22s 1ms/step - loss: 0.0847 - acc: 0.9692 - val_loss: 0.3641 - val_acc: 0.9268
Epoch 5/10
17636/17636 [==============================] - 22s 1ms/step - loss: 0.0831 - acc: 0.9705 - val_loss: 0.2376 - val_acc: 0.9451
Epoch 6/10
17636/17636 [==============================] - 22s 1ms/step - loss: 0.0780 - acc: 0.9715 - val_loss: 0.2059 - val_acc: 0.9420
Epoch 7/10
17636/17636 [==============================] - 22s 1ms/step - loss: 0.0648 - acc: 0.9777 - val_loss: 0.2070 - val_acc:

In [13]:
pred = model.evaluate(X_test, y_test)

5512/5512 [==============================] - 3s 608us/step


In [14]:
print (pred)

[0.21911884860523553, 0.9395863570391872]


In [15]:
y_pred = model.predict_classes(X_test)

In [16]:
print (y_test[0:10])

[1 1 0 0 0 0 1 1 1 1]


In [17]:
P = 0
N = 0
TP = 0
TN = 0
FP = 0
FN = 0

In [18]:
for i in range(len(X_test)):
    if y_pred[i] == 1:
        P+=1
        if y_pred[i] == y_test[i]:
            TP+=1
        else:
            FP+=1
    else:
        N+=1
        if y_pred[i] == y_test[i]:
            TN+=1
        else:
            FN+=1

In [20]:
print (TP+FN)

2797


In [19]:
print (TP, TN, FP, FN)

2656 2523 192 141


In [21]:
Accuracy = (TP+TN)/(P+N)

In [22]:
print (Accuracy)

0.9395863570391872


In [23]:
sensitivity = TP/(TP+FN)

In [24]:
print (sensitivity)

0.9495888451912764


In [25]:
specificity = TN/(TN+FP)

In [26]:
print (specificity)

0.9292817679558011


In [27]:
f1_score = 2*TP/((2*TP)+FP+FN)
print (f1_score)

0.9410097431355181
